In [ ]:
IMG_SIZE = (224, 224, 3)

In [ ]:
from keras import applications as kapps
from skimage.io import imread
import ipywidgets as ipw
from ipywebrtc import ImageRecorder, CameraStream
import PIL, io
import numpy as np

In [ ]:
if True:
    camera = CameraStream.facing_user(audio=False)
else:
    camera = CameraStream.facing_environment(audio=False)
image_recorder = ImageRecorder(stream=camera)

In [ ]:
mn_model = kapps.MobileNetV2(input_shape = IMG_SIZE, include_top=False)
prep_func = kapps.mobilenet_v2.preprocess_input

In [ ]:
image = ipw.Image()
output = ipw.Output()
tool_options = [('train', 1), ('search', 0)]
tool_mode = ipw.ToggleButtons(options=tool_options)

@output.capture()
def update_image(change):
    # turn into nparray
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_in = im_in.resize(IMG_SIZE[:2])
    im_array = np.array(im_in)[...,:3] # no alpha
    im_array = np.expand_dims(im_array.astype(np.float32), 0)
    prep_array = prep_func(im_array)
    mn_model.predict(prep_array)
    print(tool_mode.value)

image_recorder.image.observe(update_image, 'value')
tool_mode.observe(update_image, 'value')
ipw.VBox([tool_mode, camera, ipw.HBox([image_recorder, image]), output])